In [ ]:
import requests
import os
from zipfile import ZipFile
import io
from bs4 import BeautifulSoup
from google.cloud import storage
from google.cloud.exceptions import Conflict

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/kaggle/input/gcloud-key/service-key-gcloud.json'

bucket_name = 'cnpj-data'

client = storage.Client()

def create_bucket():
    location = 'SOUTHAMERICA-EAST1'

    try:
        # Attempt to create a new bucket
        bucket = client.create_bucket(bucket_name,location=location)
        print(f'Bucket {bucket_name} created in {bucket_location}.')
    except Conflict:
        print(f"Bucket '{bucket_name}' already exists.")

create_bucket()

In [ ]:
bucket = client.get_bucket(bucket_name)
folder_path = 'compressed/'

existing_files = [blob.name for blob in bucket.list_blobs()]

url_list = ["http://200.152.38.155/CNPJ/",
            "http://200.152.38.155/CNPJ/regime_tributario/"]

for url in url_list:
    response = requests.get(url)
    soup = BeautifulSoup(response.text,'html.parser')
    file_list = soup.find_all('a',href= lambda href: href.endswith('.zip'))

    for file in file_list:
        file_name = file['href']
        file_url = url + '/' + file_name
        if folder_path + file_name in existing_files:
            print(f"Skipping upload for file: {file_name}")
            continue

        # Perform the upload
        blob = bucket.blob(folder_path + file_name)
        response = requests.get(url + '/' + file_name)
        blob.upload_from_string(response.content)
        print(f"Uploaded {file_name}")

In [ ]:
# Extract zip files to a new folder

bucket_name = 'cnpj-data'
compressed_folder = 'compressed/'
destination_folder = 'extracted/'

storage_client = storage.Client()

bucket = storage_client.bucket(bucket_name)
blobs = bucket.list_blobs(prefix=compressed_folder)

for blob in blobs:
    if blob.name.endswith('.zip'):
        zip_data = io.BytesIO(blob.download_as_string())

        with ZipFile(zip_data,'r') as myzip:
            for contentfilename in myzip.namelist():
                contentfile = myzip.read(contentfilename)
                destination_path = os.path.join(destination_folder, contentfilename)
                destination_blob = bucket.blob(destination_path)
                destination_blob.upload_from_string(contentfile)
                print(f'Extracted {contentfilename}')